In [1]:
import pyodbc
import pandas as pd

In [2]:
sql_driver = 'DRIVER={ODBC Driver 13 for SQL Server};'
sql_server = 'SERVER=sage;'
sql_db = 'DATABASE=BUYS;'
sql_UID = 'Trusted_Connection=yes;'

cnxn = pyodbc.connect(sql_driver + sql_server + sql_db + sql_UID)

def fetch_data(q, cnxn):
    df = pd.read_sql(sql=q, con=cnxn)
    return df

In [3]:
StartDate = '1/1/2018'
EndDate = '12/31/2018'

query_gradedist = f'''
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
DECLARE @StartDate DATE = '{StartDate}'
DECLARE @EndDate DATE = '{EndDate}'

--Create table for buy item grades
CREATE TABLE #ItemGrades
	(
	Grade INT,
	RangeStart FLOAT,
	RangeEnd FLOAT
	)

--Insert values into buy item grade table
--All treatment of range values should be inclusive of RangeStart and exclusive of RangeEnd
INSERT INTO #ItemGrades
	(Grade, RangeStart, RangeEnd)
VALUES
	(1, 0, 8),
	(2, 8, 31),
	(3, 31, 91),
	(4, 91, 180),
	(5, 180, 10000)


SELECT
	ItemCode,
	CatalogID,
	First_RecordDate,
	LastEventType,
	CASE 
		WHEN Total_Scan_Count > 0
		THEN Days_Salable_Scanned
		ELSE Days_Salable_Priced
		END	[Days_Salable]
INTO #LifeCycle
FROM Buy_Analytics..ItemCode_LifeCycle 
WHERE ItemCode NOT IN (
			SELECT
				ItemCode
			FROM Buy_Analytics..ItemCode_LifeCycle  
			WHERE		
					 --The conditions below, taken together, indicate an item that was trashed within the first week of being priced - likely indicating a corrected pricing mistake
						LastEventType = 4
                        AND LifeCycle_Complete = 1
                        AND (CASE 
							WHEN Total_Scan_Count > 0
							THEN Days_Salable_Scanned
							ELSE Days_Salable_Priced
							END) <= 7
			)
		AND	First_RecordDate >= @StartDate
		AND First_RecordDate <= @EndDate
		AND CatalogID > 0


--Get the average days salable priced and quantity purchased in time period for each catalog ID in the lifecycle tables.
SELECT 
	lc.CatalogID,
	AVG(lc.Days_Salable) [avg_DaysInStock]
INTO #DaysInStock
FROM #LifeCycle lc
GROUP BY lc.CatalogID


--Assign a grade to each item in the #DaysInStock table using the #ItemGrades table.
SELECT 
	dis.CatalogID,
	dis.avg_DaysInStock,
	ig.Grade 
INTO #GradedCatalog
FROM #DaysInStock dis
	INNER JOIN #ItemGrades ig 
		ON	dis.avg_DaysInStock >= ig.RangeStart
		AND dis.avg_DaysInStock < ig.RangeEnd


SELECT 
	gc.Grade [Grade],
	ROUND(lc.Days_Salable, 0) [Days_Salable],
	COUNT(gc.CatalogID) [qty_Priced],
    SUM(spi.Price) [amt_Priced]
INTO #Priced
FROM #LifeCycle lc
	INNER JOIN #GradedCatalog gc
		ON lc.CatalogID = gc.CatalogID
    INNER JOIN ReportsData..SipsProductInventory spi
        ON lc.ItemCode = spi.ItemCode
GROUP BY gc.Grade, ROUND(lc.Days_Salable, 0)
ORDER BY Grade ASC, Days_Salable ASC


SELECT 
	gc.Grade [Grade],
	ROUND(lc.Days_Salable, 0) [Days_Salable],
	SUM(ssh.RegisterPrice) [amt_PricedSold],
	COUNT(gc.CatalogID) [qty_PricedSold],
	--COUNT(DISTINCT gc.CatalogID) [ttl_DSSdist]
INTO #PricedSold
FROM #LifeCycle lc
	INNER JOIN #GradedCatalog gc
		ON lc.CatalogID = gc.CatalogID
	INNER JOIN ReportsData..SipsSalesHistory ssh
		ON lc.ItemCode = ssh.SipsItemCode
WHERE lc.LastEventType = 5 --LastEventType = 5 indicates the item was sold.
GROUP BY gc.Grade, ROUND(lc.Days_Salable, 0)
ORDER BY Grade ASC, Days_Salable ASC

SELECT
	p.Grade,
	p.Days_Salable,
	p.qty_Priced,
    p.amt_Priced,
	ps.qty_PricedSold,
	ps.amt_PricedSold
FROM #Priced p
	INNER JOIN #PricedSold ps
		ON p.Grade = ps.Grade
		AND p.Days_Salable = ps.Days_Salable
ORDER BY p.Grade, p.Days_Salable


DROP TABLE #ItemGrades
DROP TABLE #LifeCycle
DROP TABLE #DaysInStock
DROP TABLE #GradedCatalog
DROP TABLE #Priced
DROP TABLE #PricedSold
'''

In [4]:
df = fetch_data(query_gradedist, cnxn)
cnxn.close()


DatabaseError: Execution failed on sql '
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
DECLARE @StartDate DATE = '1/1/2018'
DECLARE @EndDate DATE = '12/31/2018'

--Create table for buy item grades
CREATE TABLE #ItemGrades
	(
	Grade INT,
	RangeStart FLOAT,
	RangeEnd FLOAT
	)

--Insert values into buy item grade table
--All treatment of range values should be inclusive of RangeStart and exclusive of RangeEnd
INSERT INTO #ItemGrades
	(Grade, RangeStart, RangeEnd)
VALUES
	(1, 0, 8),
	(2, 8, 31),
	(3, 31, 91),
	(4, 91, 180),
	(5, 180, 10000)


SELECT
	ItemCode,
	CatalogID,
	First_RecordDate,
	LastEventType,
	CASE 
		WHEN Total_Scan_Count > 0
		THEN Days_Salable_Scanned
		ELSE Days_Salable_Priced
		END	[Days_Salable]
INTO #LifeCycle
FROM Buy_Analytics..ItemCode_LifeCycle 
WHERE ItemCode NOT IN (
			SELECT
				ItemCode
			FROM Buy_Analytics..ItemCode_LifeCycle  
			WHERE		
					 --The conditions below, taken together, indicate an item that was trashed within the first week of being priced - likely ind

In [ ]:
df.to_csv('./optimalturnrate_2018.csv')